# 4. 신경망 학습
학습을 통해 가중치 매개변수의 최적값을 자동으로 획득해보자.   
신경망을 학습할 수 있게 해주는 지표인 손실 함수를 통해, 손실 함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는 것이 학습의 목표이다.   
함수의 기울기를 활용하는 경사법을 알아보자.

# 4.1 데이터에서 학습한다!
신경망은 데이터를 보고 가중치 매개변수의 값을 자동으로 결정한다.   
신경망 학습과 MNIST 데이터셋의 손글씨 숫자를 학습하는걸 코드로 구현해보자.

### 4.1.1 데이터 주도 학습
머신러닝은 데이터가 생명이다.   
데이터에서 패턴을 발견해 답을 찾는다.

보통 문제를 해결하기 위해, 사람이 생각하고 답을 찾는게 일반적이다.   
그리고 사람의 경험과 직관을 단서로 시행착오를 거듭하며 일을 진행한다.   
하지만 기계학습은 사람의 개입을 최소화하고 데이터로부터 패턴을 찾으려 시도한다.   
특히 신경망과 딥러닝은 기존 기계학습보다 사람의 개입을 더욱 배제한다.

사람이 숫자를 분류하는 프로그램을 설계하려면, 그 안에 숨은 규칙성을 명확한 로직으로 풀기 어렵다.   
그래서 신경망은 이미지에서 __특징(feature)__을 추출하고 특징의 패턴을 기계학습 기술로 학습한다.   
여기서 특징은 입력 데이터에서 중요한 데이터를 정확하게 추출할 수 있도록 설계된 변환기를 가리킨다.

기계학습은 모아진 데이터로부터 기계가 규칙을 찾아낸다.   
하지만 이미지를 벡터로 변환할 때 사용하는 특징은 여전히 사람이 설계해야 한다.   
이는 문제에 적합한 특징을 사용하지 않으면 좋은 결과를 얻을 수 없음을 뜻한다.   
즉 기계학습도 사람이 적절한 특징을 생각해야한다.   

신경망은 이미지에 포함된 중요한 특징까지 기계가 스스로 학습하게 된다.   
신경망의 이점으로 모든 문제를 같은 맥락에서 풀 수 있고, 주어진 문제의 패턴을 발견하려 시도한다.   
즉, 모든 문제를 주어딘 데이터 그대로를 입력으로 활용해 end-to-end로 학습할 수 있다.

### 4.1.2 훈련 데이터와 시험 데이터
기계학습은 범용 능력(아직 보지 못한 데이터)을 제대로 획득하는 것이 최종 목표이다.   
그래서 데이터셋을 훈련 데이터와 시험 데이터를 나눠서 평가한다.   
하지만 데이터셋 하나로만 학습과 평가를 진행하면 수중의 데이터셋만 지나치게 최적화된 오버피팅 현상이 나올 수도 있다.

# 4.2 손실 함수
신경망에서 현재의 상태를 하나의 지표로 표현한다.   
그리고 그 지표를 가장 좋게 만들어주는 가중치 매개변수의 값을 탐색해야한다.   
신경망 학습에서 사용하는 지표는 __손실 함수(loss function)__이다.   
일반적으로 오차제곱합, 교체 엔트로피 오차를 사용한다.

### 4.2.1 오차제곱합
__오차제곱합(sum of squares for error, SSE)__을 알아보자.   
$E = {{1}\over{2}}\sum\limits_k(y_k-t_k)^2$   
$y_k$는 신경망의 출력, $t_k$는 정답 레이블, $k$는 데이터의 차원 수를 나타낸다.

여기서 y는 소프트맥스의 출력으로 확률로 해석하자.   
그리고 t는 정답만 1로 나타내는 원-핫 인코딩 형태이다.

In [1]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [2]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [3]:
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [4]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

sum_squares_error(np.array(y), np.array(t))

0.5975

신경망이 추정한 값과 정답이 일치한 상황과 그렇지 않은 상황에 대해 SSE 값을 출력해봤다.

### 4.2.2 교차 엔트로피 오차
손실 함수로 __교차 엔트로피 오차(cross entropy error, CEE)__를 많이 사용하므로 확인해보자.

$E = -\sum\limits_kt_klogy_k$

여기서 log는 밑이 $e$인 자연로그($log_e$)이다.   
$y_k$는 신경망의 출력(추정값), $t_k$는 정답 레이블이고 원-핫 인코딩이다.   
그래서 실질적으로 정답일 때 추정($t_k$가 1일때 $y_k$)의 자연로그를 계산하게 된다.   
예를들면 정답 레이블에 맞는 신경망의 출력 $y_k$가 0.6이면 교차 엔트로피 오차는 -log0.6 = 0.51이 된다.   
같은 조건에서 신경망 출력이 0.1이면 -log0.1 = 2.30이 된다.   

![](https://support.minitab.com/ko-kr/minitab/18/naturallog_scatterplot.png)   
자연로그의 그래프를 확인해보면 x가 1일때 y가 0이되고, x가 0에 가까워질수록 y의 값은 점점 작아진다.

In [5]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

np.log() 함수에 0을 넣으면 마이너스 무한대인 -inf가 나와 계산을 진행할 수 없기에, 아주 작은 값인 delta를 넣어줬다.

In [6]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [7]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 4.2.3 미니배치 학습
기계학습 문제는 훈련 데이터에 대한 손실 함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾아낸다.   
즉, 훈련 데이터 개수만큼 이들의 손실 함수 값들의 합을 지표로 삼는다.   

훈련 데이터 모두에 대한 손실 함수의 합을 구하는 방법을 알아보자.   
$E = -{{1}\over{N}}\sum\limits_n\sum\limits_kt_{nk}logy_{nk}$ 

데이터가 $N$개이고 $t_{nk}$는 $n$번째 데이터의 $k$번째 값을 의미한다.   
그리고 $N$으로 나누어 정규화를 하고, $N$으로 나눔으로써 __평균 손실 함수__를 구하는 것이다.

MNIST의 데이터셋은 훈련 데이터가 60000개였다.   
모든 데이터를 대상으로 손실 함수의 합을 구하려면 시간이 걸린다.   
그래서 데이터의 일부를 추려 전체의 근사치로 이용할 수 있다.   
이 일부를 __미니 배치__라고 한다.   
즉, 6만장의 훈련 데이터 중에서 100장을 무작위로 뽑아 학습하는 것이다.

### MNIST dataset 불러오기

In [8]:
# coding: utf-8
try:
    import urllib.request
except ImportError:
    raise ImportError('You should use Python 3.x')
import os.path
import gzip
import pickle
import os
import numpy as np


url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

dataset_dir = os.path.dirname('/content/drive/Othercomputers/MacBook_Air/Deep_Learning_from_Scratch/V_01/dataset')
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784


def _download(file_name):
    file_path = dataset_dir + "/" + file_name

    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0"}
    request = urllib.request.Request(url_base+file_name, headers=headers)
    response = urllib.request.urlopen(request).read()
    with open(file_path, mode='wb') as f:
        f.write(response)
    print("Done")

def download_mnist():
    for v in key_file.values():
       _download(v)

def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")

    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")

    return data

def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done!")

def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1

    return T


def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """MNISTデータセットの読み込み

    Parameters
    ----------
    normalize : 画像のピクセル値を0.0~1.0に正規化する
    one_hot_label :
        one_hot_labelがTrueの場合、ラベルはone-hot配列として返す
        one-hot配列とは、たとえば[0,0,1,0,0,0,0,0,0,0]のような配列
    flatten : 画像を一次元配列に平にするかどうか

    Returns
    -------
    (訓練画像, 訓練ラベル), (テスト画像, テストラベル)
    """
    if not os.path.exists(save_file):
        init_mnist()

    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)

    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0

    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])

    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label'])


if __name__ == '__main__':
    init_mnist()


Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [9]:
import sys, os
# sys.path.append(os.pardir)
import numpy as np
# from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [10]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [11]:
np.random.choice(60000, 10)

array([25501, 56702, 18346, 33430, 51380, 43700,  8164, 10024, 57099,
       10590])

훈련 데이터에서 무작위로 batch_size만큼 꺼내올 수 있다.   
np.random.choice()로 랜덤으로 선택해 인덱스로 사용하여 미니배치를 뽑아내면 된다.

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기
교차 엔트로피 오차에서 배치 데이터를 구현할 수 있게 해보자.   
예측한 y의 값이 1차원인 데이터 하나당 교차 엔트로피 오차를 구하는 경우에는 reshape로 데이터의 형상을 바꿔준다.   
그리고 배치 크기로 나눠 정규화하고 이미지 1장당 평균의 교차 엔트로피 오차를 계산한다.

In [15]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

정답이 원-핫 인코딩이 아닌 숫자 레이블로 주어졌을 때 교차 엔트로피 오차를 어떻게 구현할까?

In [16]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

이 구현에서 원-핫 인코딩일 때 t가 0인 원소는 교차 엔트로피 오차도 0이므로, 그 계산은 무시해도 좋다는 것이 핵심이다.   
이는 정답에 해당하는 신경망의 출력만으로 교차 엔트로피 오차를 계산할 수 있다.   
그래서 원-핫 인코딩시 `t*np.log(y)`를 레이블 표현은 `np.log(y[np.arange(batch_size), t])`로 구현한다.

`np.log(y[np.arange(batch_size), t])`에서 np.arange(batch_size)는 0부터 batch_size-1까지 배열을 생성한다.   
t에는 레이블이 [2,7,0,9,4]와 같이 저장되어 있으므로 `y[np.arange(batch_size], t`는 각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출한다.   
(예를들자면 `[y[0,2], y[1,7], y[2,0], y[3,9], y[4,4]`인 넘파이 배열을 생성한다.)

### 4.2.4 왜 손실 함수를 설정하는가?
궁극적인 목표는 높은 정확도를 끌어내는 매개변수 값을 찾는 것이다.   
그렇다면 왜 정확도 지표를 두고 손실 함수의 값을 택할까?

신경망 학습에서 __미분__의 역할이 있기 때문이다.   
신경망 학습에서 최적의 매개변수(가중치, 편향)을 탐색할 때 손실 함수의 값을 가능한 작게 하는 매개변수 값을 찾는다.   
이때 매개변수의 기울기를 미분을 통해 계산하고, 그 미분 값을 단서로 매개변수의 값을 갱신한다.

예를들어 가중치 매개변수의 손실 함수의 미분이란?   
> 가중치 매개변수의 값을 아주 조금 변화 시켰을 때, 손실 함수가 어떻게 변하는지 볼 수 있다.

만약 미분 값이 음수면 가중치 매개변수를 양의 방향으로 변화 시켜 손실 함수의 값을 줄일 수 있다.   
반대로 미분 값이 양수면 가중치 매개변수를 음의 방향으로 변화시켜 손실 함수의 값을 줄일 수 있다.   
그러나 미분 값이 0이면 가중치 매개변수를 어느 쪽으로 움직여도 손실 함수의 값은 줄어들지 않는다.   
그래서 가중치 매개변수의 갱신은 거기서 멈춘다.

정확도를 지표로 삼으면 미분 값이 대부분의 장소에서 0이 되어 매개변수를 갱신할 수 없게 된다.   
이를 구체적인 예를 통해 알아보자.   
한 신경망이 100장의 훈련 데이터 중 32장을 올바로 인식한다면 정확도는 32%이다.   
정확도가 지표였다면 가중치 매개변수의 값을 조금 조절해도 정확도는 32%이다.   
즉, 매개변수를 약간만 조정해서는 정확도가 개선되지 않는다.   
만약 개선된다 하더라도 그 값은 32.0123%와 같은 연속적인 변화보다는 33%, 34%처럼 불연속적인 값으로 바뀌어버린다.

손실 함수를 지표로 삼으면 어떻게 바뀔까?   
현재 손실 함수의 값은 0.92543... 같은 수치이다.   
구리고 매개변수의 값이 조금 변하면 그에 반응하여 손실 함수의 값도 0.93432... 처럼 연속적으로 변화한다.

정리하자면 정확도는 매개변수의 사소한 변화에 거의 반응을 보이지 않고, 반응이 있더라도 그 값이 불 연속적이다.   
이는 계단 함수를 활성화 함수로 사용하지 않는 이유와 동일하다.   
만약 활성화 함수로 계단 함수를 사용하면 신경망 학습이 잘 이뤄지지 않는다.   
계단 함수의 미분은 0이 아닌 장소에서 0이다.   

계단 함수는 한순간만 변화를 일으키지만, 시그모이드 함수의 미분은 연속적으로 변하고 곡선의 기울기도 연속적으로 변한다.   
즉 시그모이드 함수의 미분은 어느 장소라도 0이 되지는 않는다.   
신경망 학습에서 중요한 성질로, 기울기가 0이 되지 않아 신경망이 올바르게 학습할 수 있다.

# 4.3 수치 미분

### 4.3.1 미분

### 4.3.2 수치 미분의 예

### 4.3.3 편미분

# 4.4 기울기

### 4.4.1 경사법(경사 하강법)

### 4.4.2 신경망에서의 기울기

# 4.5 학습 알고리즘 구현하기

### 4.5.1 2층 신경망 클래스 구현하기

### 4.5.2 미니배치 학습 구현하기

### 4.5.3 시험 데이터로 평가하기